In [1]:
import pyspark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1754380185469_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, input_file_name, regexp_extract, isnull, isnan, lit, hour, to_timestamp, to_date, when, concat_ws, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Start Spark Session
spark = SparkSession.builder.appName("WeatherFileAnalysis").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Paths
base_path = "s3://allweatherdatastatelatlong/wd2statelatlong/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_all = spark.read.option("header", True).parquet(base_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_all.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['_c0', 'date', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature', 'precipitation', 'rain', 'snowfall', 'snow_depth', 'pressure_msl', 'surface_pressure', 'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high', 'wind_speed_10m', 'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m', 'wind_gusts_10m', 'file_only']

In [7]:
# Extract city from file path
df_cleaned = df_all.withColumn(
    "city",
    regexp_extract(col("file_only"), r"/([^/]+?)(?:_\d+)?\.csv$", 1)
)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_cleaned.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['_c0', 'date', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature', 'precipitation', 'rain', 'snowfall', 'snow_depth', 'pressure_msl', 'surface_pressure', 'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high', 'wind_speed_10m', 'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m', 'wind_gusts_10m', 'file_only', 'city']

In [9]:
df_cleaned[["city", "file_only"]].head(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(city='Devipattinam', file_only='s3://wd1public/wd1publicdata/Devipattinam.csv')]

In [10]:
from pyspark.sql.functions import regexp_replace, initcap

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_with_city = df_cleaned.withColumn("city", regexp_replace("city", "%20", " ")) \
               .withColumn("city", initcap("city")) \
               .withColumn("city", regexp_replace("city", " ", ""))  # Remove spaces

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_with_city.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['_c0', 'date', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature', 'precipitation', 'rain', 'snowfall', 'snow_depth', 'pressure_msl', 'surface_pressure', 'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high', 'wind_speed_10m', 'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m', 'wind_gusts_10m', 'file_only', 'city']

In [13]:
# Fill specific null values
df_filled = df_with_city.fillna({"snow_depth": 0})

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# ====================  CITY CLASSIFICATION ====================
city_file_counts = df_with_city.groupBy("city") \
    .agg(F.countDistinct("file_only").alias("file_count"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
multi_city_list = city_file_counts.filter("file_count > 1").select("city").rdd.flatMap(lambda x: x).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
single_city_list = city_file_counts.filter("file_count = 1").select("city").rdd.flatMap(lambda x: x).collect()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# ====================  UTILITY FUNCTION ====================
def create_12hr_agg(df_input):
    df = df_input.withColumn("datetime_ts", to_timestamp("date")) \
        .withColumn("hour", hour("datetime_ts")) \
        .withColumn("day_part", when(col("hour") < 12, "AM").otherwise("PM")) \
        .withColumn("date", to_date(col("datetime_ts"))) \
        .withColumn("new_datetime", concat_ws(" ",
                                              col("date"),
                                              when(col("day_part") == "AM", lit("00:00:00"))
                                                 .otherwise(lit("12:00:00"))))

    key_metrics = [
        "temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature",
        "precipitation", "rain", "snowfall", "snow_depth", "pressure_msl", "surface_pressure",
        "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high",
        "wind_speed_10m", "wind_speed_100m", "wind_direction_10m", "wind_direction_100m",
        "wind_gusts_10m"
    ]

    agg_exprs = [avg(col(m)).alias(m) for m in key_metrics]
    df_12hr = df.groupBy("new_datetime", "city").agg(*agg_exprs)
    return df_12hr.select("new_datetime", "city", *key_metrics).orderBy("new_datetime", "city")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# ==================== MULTI-FILE MERGING ====================
df_multi = df_filled.filter(col("city").isin(multi_city_list))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Merge multiple files by date + city
numeric_cols = [c for c, t in df_multi.dtypes if t in ['double', 'int', 'float', 'long'] and c not in ['date', 'city']]
string_cols = [c for c, t in df_multi.dtypes if t in ['string'] and c not in ['date', 'city']]

merge_exprs = (
    [F.avg(col(c)).alias(c) for c in numeric_cols] +
    [F.first(col(c), ignorenulls=True).alias(c) for c in string_cols]
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_merged = df_multi.groupBy("date", "city").agg(*merge_exprs)
df_12hr_multi = create_12hr_agg(df_merged)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:

# ==================== SINGLE FILE CITIES ====================
df_single = df_filled.filter(col("city").isin(single_city_list))
df_12hr_single = create_12hr_agg(df_single)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# ==================== FINAL UNION ====================
df_12hr_final = df_12hr_multi.unionByName(df_12hr_single).orderBy("new_datetime", "city")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
unique_city_count = df_12hr_final.select("city").distinct().count()
print(f"Total unique cities: {unique_city_count}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total unique cities: 1527

In [42]:
df_mapping = spark.read.option("header", True).csv("s3://citystatelatlong/city_state_latlong/merged_city_state_lat_long.csv") \
    .withColumn("latitude", col("latitude").cast("double")) \
    .withColumn("longitude", col("longitude").cast("double")) \
    .withColumn("city", regexp_replace("city", "%20", " ")) \
    .withColumn("city", initcap("city")) \
    .withColumn("city", regexp_replace("city", " ", "")) \
    .withColumn("state", initcap("state")) 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df_mapping.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['city', 'state', 'latitude', 'longitude']

In [26]:
df_mapping[["city", "state",'latitude','longitude']].head(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(city='Abbigeri', state='Karnataka', latitude=15.5862, longitude=75.7502)]

In [43]:
df_mapped = df_12hr_final.join(df_mapping.select("city", "state", "latitude", "longitude"), on="city", how="left")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df_mapped.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['city', 'new_datetime', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m', 'apparent_temperature', 'precipitation', 'rain', 'snowfall', 'snow_depth', 'pressure_msl', 'surface_pressure', 'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high', 'wind_speed_10m', 'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m', 'wind_gusts_10m', 'state', 'latitude', 'longitude']

In [30]:
from pyspark.sql.functions import col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
df_null_mapped = df_mapped.filter(
    col("state").isNull() | col("latitude").isNull() | col("longitude").isNull()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
null_city_count = df_null_mapped.select("city").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
null_city_count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [48]:
df_mapped.select("state").distinct().show(50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|               state|
+--------------------+
|              Kerala|
|             Manipur|
|                 Goa|
|    Himachal Pradesh|
|             Gujarat|
|              Sikkim|
|               Delhi|
|           Meghalaya|
|Andaman And Nicob...|
|           Karnataka|
|              Punjab|
|             Mizoram|
|           Jharkhand|
|           Telangana|
|             Tripura|
|            Nagaland|
|        Chhattisgarh|
|           Rajasthan|
|         Maharashtra|
|               Bihar|
+--------------------+
only showing top 20 rows

In [49]:
df_mapped.filter("state IS NULL").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [52]:
df_mapped.write \
    .mode("overwrite") \
    .partitionBy("state") \
    .parquet("s3://wd2partioned/wd3statewise/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
df_mapped.select("city").distinct().count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1527

In [54]:
df_mapped.select("state").distinct().count()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

32